In [1]:
import sys
path = '/Users/apple/desktop/Yangze_Investment/Task8_Reproduce_Report/Task8_20200304_2022'
sys.path.append(f'{path}/self_defined')

In [2]:
from calc_inv_data import calc_inv_data
from process_total_inv import process_total_inv
from generate_inv_signal_seprank import generate_inv_signal_seprank
from generate_inv_signal_unirank import generate_inv_signal_unirank
from generate_inv_signal_allinone import generate_inv_signal_allinone
from generate_ret_fnv import generate_ret_fnv

In [11]:
%%time

# import sys
# import math
# # import xlwings
# import numpy as np
import pandas as pd
from yzutil import YzDataClient


# initiate data api obj
yz = YzDataClient('bruce@yangzeinvest.com', 'bruce123')

# load data
inv_index_tb = pd.read_csv(f'{path}/data/inventory_data_index.csv')
futures_data_df = pd.read_csv(f'{path}/data/futures_data.csv', index_col=0)

# set parameters
SIG_THRESHOLD = 0.2

mode = 'ret_annualized_pair'

shift_inv, shift_sig = 'shift_yes', 'shift_yes'

# rolling_list = [5]
rolling_list = [5] + [i for i in range(10, 110, 10)]

annualized_ret_df = pd.DataFrame()

# main function calls
inv_total_tradingday_df = process_total_inv(inv_index_tb, yz, shift_inv)

for rolling in rolling_list:
    
    # main function call
    signal_df = generate_inv_signal_seprank(inv_total_tradingday_df, rolling, SIG_THRESHOLD)
    
    annualized_ret_pair = generate_ret_fnv(inv_index_tb, futures_data_df, signal_df, shift_sig, mode, '2017-07-31')
    
    annualized_ret_pair_df = pd.DataFrame(annualized_ret_pair)
    
    annualized_ret_df = pd.concat([annualized_ret_df, annualized_ret_pair_df], axis=1)
    
annualized_ret_df.index = ['annualized_ret', 'annualized_ret_DeductTC']
annualized_ret_df.columns = rolling_list

annualized_ret_df

Login YzData Success
CPU times: user 2min 51s, sys: 760 ms, total: 2min 52s
Wall time: 3min 13s


,5,10,20,30,40,50,60,70,80,90,100
annualized_ret,0.0547,0.0665,0.0132,-0.0120,-0.0043,-0.0086,0.0017,-0.0063,0.0298,0.0537,0.0485
annualized_ret_DeductTC,0.0082,0.0391,-0.0046,-0.0255,-0.0161,-0.0190,-0.0082,-0.0152,0.0202,0.0457,0.0408


In [12]:
annualized_ret_df.to_csv(f'{path}/results/annualized_ret_SepRank_20170731_R5to100_df.csv')